In [2]:
import numpy as np
import pandas as pd
import os

df = pd.read_csv('MetObjects.csv')

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7,8,9,10,11,18,27,28,29,30,31,32,33,34,35,36,37,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df['Classification'].value_counts()

Prints                                                                  69260
Prints|Ephemera                                                         30033
Photographs                                                             26821
Drawings                                                                25230
Books                                                                   14685
Ceramics                                                                13332
Paintings                                                               11038
Textiles-Woven                                                          10995
Photographs|Ephemera                                                    10940
Glass                                                                    8838
Negatives                                                                6460
Vases                                                                    5050
Textiles-Laces                                                  

In [97]:
public_domain = df.loc[df['Is Public Domain'] == True]
buckets = public_domain.groupby('Object Name').filter(lambda x: len(x) >= 1000)
classes = buckets['Object Name']
downloaded = buckets.loc[:100888]
print(len(downloaded))
#for idx, label in selected_labels.iteritems():
#    print(idx)
#for idx, label in classes.iteritems():
    #print("{} {}".format(idx, label))
#buckets['Object Name'].value_counts()
#links = buckets['Link Resource']
#links.head()
#links.loc[37:]
#buckets = df.loc[df['']]
#buckets.iloc[37:]
#buckets['Link Resource'].index
#objects = public_domain['Object Name'].value_counts()

17419


In [89]:
downloaded.loc[downloaded['Object Name'] == 'Vase']
d = downloaded.index.tolist()

[34,
 36,
 37,
 117,
 118,
 119,
 137,
 163,
 244,
 282,
 381,
 410,
 411,
 412,
 413,
 414,
 415,
 416,
 417,
 418,
 419,
 420,
 427,
 428,
 429,
 430,
 431,
 445,
 464,
 521,
 525,
 526,
 527,
 528,
 529,
 530,
 531,
 532,
 533,
 534,
 537,
 538,
 539,
 541,
 542,
 543,
 544,
 548,
 552,
 554,
 555,
 556,
 557,
 558,
 559,
 560,
 561,
 562,
 563,
 564,
 565,
 566,
 567,
 568,
 569,
 570,
 571,
 572,
 575,
 576,
 583,
 584,
 585,
 593,
 594,
 595,
 596,
 597,
 598,
 599,
 600,
 601,
 602,
 604,
 605,
 607,
 608,
 609,
 610,
 611,
 614,
 615,
 616,
 617,
 620,
 622,
 623,
 624,
 626,
 628,
 629,
 630,
 631,
 632,
 633,
 634,
 638,
 639,
 640,
 641,
 642,
 643,
 644,
 645,
 646,
 647,
 649,
 650,
 651,
 652,
 653,
 654,
 655,
 656,
 657,
 658,
 659,
 660,
 661,
 662,
 663,
 664,
 665,
 666,
 667,
 668,
 669,
 670,
 671,
 674,
 675,
 678,
 680,
 681,
 682,
 683,
 684,
 685,
 687,
 688,
 689,
 690,
 691,
 692,
 693,
 694,
 695,
 696,
 697,
 698,
 700,
 701,
 702,
 703,
 705,
 706,
 708,
 

In [95]:
import random
# get the top 10 classes
top_10_labels = classes.value_counts()[:10]
# convert them to a list (https://stackoverflow.com/a/41456363/1221477)
top_10_labels = top_10_labels.axes[0].tolist()
training = {}
testing = {}
for label in top_10_labels:
    # get downloaded images matching this class
    downloaded_imgs = downloaded.loc[downloaded['Object Name'] == label]
    downloaded_imgs_indexes = downloaded_imgs.index.tolist()
    random.shuffle(downloaded_imgs_indexes)
    num_imgs = len(downloaded_imgs_indexes)
    if len(downloaded_imgs_indexes) >= 1000:
        num_imgs = 1000
    training_set_size = int(0.9 * num_imgs)
    sampled_downloaded_imgs_indexes = downloaded_imgs_indexes[:num_imgs]
    training[label] = sampled_downloaded_imgs_indexes[:training_set_size]
    testing[label] = sampled_downloaded_imgs_indexes[training_set_size:num_imgs]

In [44]:
from scraper.met_scraper import get_starting_id
starting_id = get_starting_id()
downloaded = buckets.loc[:100888]
downloaded['Object Name'].value_counts()
#print(downloaded)

Print         3483
Piece         3438
Drawing       2192
Vase          1599
Painting      1351
Figure        1182
Bowl           928
Dish           719
Plate          649
Medal          512
Bottle         454
Jar            429
Ornament       185
Panel          171
Fragment       117
Photograph      10
Name: Object Name, dtype: int64

In [37]:
with open("labels.txt", "w") as j:
    for index, row in downloaded.iterrows():
        j.write("{}\n".format(row['Object Name']))

In [134]:
pieces = public_domain['Link Resource']
print(pieces.head())
for artwork_id, link in pieces[:3].iteritems():
    print(id, link)

32    http://www.metmuseum.org/art/collection/search/33
33    http://www.metmuseum.org/art/collection/search/34
34    http://www.metmuseum.org/art/collection/search/35
35    http://www.metmuseum.org/art/collection/search/36
36    http://www.metmuseum.org/art/collection/search/37
Name: Link Resource, dtype: object
34 http://www.metmuseum.org/art/collection/search/33
34 http://www.metmuseum.org/art/collection/search/34
34 http://www.metmuseum.org/art/collection/search/35


In [72]:
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import requests
import re

response = requests.get(link, timeout=5)
page_content = BeautifulSoup(response.content, "html.parser")
download_button = page_content.find(class_='utility-menu__item utility-menu__item--download')

<class 'bytes'>


In [89]:
href = download_button.a['href']
img_link = re.search('https\S+.\w', href).group(0)
extension = re.search('.(\w+)$', img_link).groups()[0]
extension

'jpg'

In [59]:
response = requests.get(img_link, stream=True, timeout=5)

In [60]:
with open("images/{}.{}".format(artwork_id, extension), 'wb') as out_file:
    response.raw.decode_content = True
    shutil.copyfileobj(response.raw, out_file)
del response

NameError: name 'artwork_id' is not defined

In [1]:
df.info

NameError: name 'df' is not defined